In [1]:
import os
import shutil
import torch
from torchvision import models, transforms
from PIL import Image


In [4]:

# Define paths
base_dir = 'test/image'
cloth_dir = 'test/classified/cloth'
person_dir = 'test/classified/person'

# Ensure target folders exist
os.makedirs(cloth_dir, exist_ok=True)
os.makedirs(person_dir, exist_ok=True)
# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the pre-trained Faster R-CNN model and move it to the device
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)
model.eval()

transform = transforms.Compose([
    transforms.ToTensor()
])


Using device: cuda


In [5]:
def classify_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)  # Move input to GPU
    
    with torch.no_grad():
        outputs = model(image_tensor)
    
    # Check if "person" is detected
    for box, label, score in zip(outputs[0]['boxes'], outputs[0]['labels'], outputs[0]['scores']):
        if label.item() == 1 and score.item() > 0.5:  # COCO label 1 is 'person'
            return "person"
    return "cloth"

# Segregate only the first 1000 images
count = 0
for filename in os.listdir(base_dir):
    file_path = os.path.join(base_dir, filename)
    if os.path.isfile(file_path):
        label = classify_image(file_path)
        target_dir = cloth_dir if label == "cloth" else person_dir
        shutil.move(file_path, target_dir)
        print(f"Moved {filename} to {label}")
        
        count += 1
        if count >= 1000:
            print("Classified 1000 images. Stopping.")
            break

Moved 001062.jpg to person
Moved 001063.jpg to person
Moved 001064.jpg to person
Moved 001065.jpg to person
Moved 001066.jpg to cloth
Moved 001067.jpg to person
Moved 001068.jpg to person
Moved 001069.jpg to person
Moved 001070.jpg to person
Moved 001071.jpg to person
Moved 001072.jpg to person
Moved 001073.jpg to cloth
Moved 001074.jpg to person
Moved 001075.jpg to person
Moved 001076.jpg to person
Moved 001077.jpg to person
Moved 001078.jpg to person
Moved 001079.jpg to cloth
Moved 001080.jpg to cloth
Moved 001081.jpg to person
Moved 001082.jpg to person
Moved 001083.jpg to person
Moved 001084.jpg to person
Moved 001085.jpg to person
Moved 001086.jpg to person
Moved 001087.jpg to cloth
Moved 001088.jpg to person
Moved 001089.jpg to person
Moved 001090.jpg to person
Moved 001091.jpg to person
Moved 001092.jpg to person
Moved 001093.jpg to person
Moved 001094.jpg to person
Moved 001095.jpg to person
Moved 001096.jpg to person
Moved 001097.jpg to person
Moved 001098.jpg to person
Moved 